In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pandasql import sqldf
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 700)

with open('output.csv') as f:
    print(f)

    
deferments = pd.read_csv("output.csv", sep = ";", encoding ="unicode_escape")
design = pd.read_csv("Designs_alle.csv", sep = ";")
tiefpumpen = pd.read_csv("Tiefpumpen.csv", sep = ";")
data = pd.read_csv("Designs_alle1.csv",  sep = ";", thousands=',')
runlife = pd.read_csv("RUNLIFE_final1.csv", sep = ";")

data_design = data.drop(data.loc[:,'TYP':'D#KUM_LASTWECHSEL_LAST_CALC'], axis = 1)
data_design = data_design.drop_duplicates()

data_tp = data.drop(data.loc[:, 'PK#ID_1':'DYNO_AUFNAHME'], axis = 1)
data_tp = data_tp.drop(columns = ['PK#ID','FARBCODE','DATUM','BEARBEITER','BEARBEITER#X_KEY','AUSFALLDATUM','BEHANDLUNGSGRUND'])
data_tp = data_tp.drop(data_tp.loc[:, 'FAE3':'FLG_FREIGABE'], axis = 1)
data_tp = data_tp.drop_duplicates()

data_raw = sqldf("SELECT runlife.*, data_tp.* \
      FROM runlife JOIN data_tp ON runlife.tp_nummer = data_tp.tiefpumpennummer \
      AND runlife.sondenkurzname_1 = data_tp.sondenkurzname")

data = sqldf("SELECT data_raw.*, data_design.* \
      FROM data_raw JOIN data_design ON data_raw.datum_1 = data_design.datum_1 \
      AND data_raw.sondenkurzname_1 = data_design.sondenkurzname_1")

data['KUM_ARBEITSTAGE'] = data['KUM_ARBEITSTAGE'].astype(float)

data = sqldf("SELECT * FROM data WHERE runlife > 0 AND runlife IS NOT NULL").drop_duplicates()

data_deferments = sqldf("SELECT data.*, deferments.* FROM data JOIN deferments ON data.tp_nummer = deferments.tpnummer")
deta_deferments = data_deferments.drop_duplicates()

data_undicht = sqldf("SELECT * FROM data_deferments WHERE abstellgrund LIKE '%Installation undicht%'")
data_tpschaden = sqldf("SELECT * FROM data_deferments WHERE abstellgrund LIKE '%Tiefpumpenfehler%'")
data_pgbruch = sqldf("SELECT * FROM data_deferments WHERE abstellgrund LIKE '%Pumpgestängebruch%'")

In [ ]:
counts, bins = np.histogram(data['RUNLIFE'], bins = 'auto')
plt.xlabel("Runlife [d]")
plt.ylabel("No. of Wells")
plt.stairs(counts, bins, fill = True)
plt.show()

In [ ]:
counts, bins = np.histogram(data_tpschaden['RUNLIFE'], bins = 20)
plt.xlabel("Runlife [d]")
plt.ylabel("No. of Wells")
plt.stairs(counts, bins, fill = True)
plt.show()

In [ ]:
counts, bins = np.histogram(data_undicht['RUNLIFE'], bins = 'auto')
plt.xlabel("Runlife [d]")
plt.ylabel("No. of Wells")
plt.stairs(counts, bins, fill = True)
plt.show()

In [ ]:
 counts, bins = np.histogram(data_pgbruch['RUNLIFE'], bins = 10)
plt.xlabel("Runlife [d]")
plt.ylabel("No. of Wells")
plt.stairs(counts, bins, fill = True)
plt.show()

In [ ]:
pumpdurchmesser = data['API_KURZBEZ_TEIL1'].str[3:]
pumpentyp = pd.get_dummies(data['API_KURZBEZ_TEIL2'])
pgzustand = pd.get_dummies(data['PG_TYP2'])
tubingcondition = pd.get_dummies(data['TUB_TYP2'])
gasanker = pd.get_dummies(data['GAS_ANKER'])
packer = pd.get_dummies(data['PACK_ANKER'])
hubzahl = data['HUBZAHL']
hublaenge = data['HUBLAENGE']
farr = data['SPEZ_FARR'].fillna(0).astype(int)
spez1 = pd.get_dummies(data['SPEZ_EQUI1'])
spez2 = pd.get_dummies(data['SPEZ_EQUI2'])
spez3 = pd.get_dummies(data['SPEZ_EQUI3'])
spez4 = pd.get_dummies(data['SPEZ_EQUI4'])
spez5 = pd.get_dummies(data['SPEZ_EQUI5'])
spez6 = pd.get_dummies(data['SPEZ_EQUI6'])

features = pd.concat([data['SONDENKURZNAME_1'], data['DATUM_1'], data['RUNLIFE'].astype(int),data['DOGLEG'],data['ABSETZTEUFE'],data['HUBZAHL'],
                      data['HUBLAENGE'], pumpdurchmesser, pumpentyp, pgzustand, 
                      tubingcondition, gasanker, packer, spez1, spez2,
                      spez3, spez4, spez5, spez6], axis = 1)


features_grouped = features.groupby(features.columns, axis=1).sum()


farr_heavycage_brass = features_grouped['TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten']
heavycage = features_grouped['!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten']

features_grouped['Tubing_pump'] = features_grouped['TH']
features_grouped['Sand_filter'] = features_grouped['Conslot Filter'] + features_grouped['Stanley Filter']
features_grouped['New_tubing'] = features_grouped['neu'] + features_grouped['gelb']
features_grouped['Spiral_rod_guides'] = features_grouped['Spiral Rod Guide 2 7/8"'] + features_grouped['Spiral Rod Guide 3 1/2"']
features_grouped['Heavy_wall_barrel'] = features_grouped['RHAC'] + features_grouped['RHBM'] + features_grouped['TH']
features_grouped['Jacket_pipe'] = features_grouped['Mantelrohr 2 7/8"'] + features_grouped['Mantelrohr 3 1/2"']
features_grouped['Gas_separator'] = features_grouped['Gasanker'] + features_grouped['Gasseparator - PCM']
features_grouped['Brass_barrel'] = features_grouped['Messingbarrel'] + farr_heavycage_brass
features_grouped['FARR_plunger'] = farr + farr_heavycage_brass
features_grouped['Heavy_cages_balls'] = heavycage + farr_heavycage_brass
features_grouped['Tubing_anchored'] = features_grouped['Steigrohranker'] + features_grouped['LSP ohne Gummi']
features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4 Ventilkugeln'] + \
                                             features_grouped['Heavy_cages_balls'])

features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4_valve_balls'])
features_grouped['Jacket_pipe'] = np.sign(features_grouped['Jacket_pipe'])
features_grouped['A6_valve_balls'] = np.sign(features_grouped['A6 Ventilkugeln'])
features_grouped['Sand_filter'] = np.sign(features_grouped['Sand_filter'].astype(int))
features_grouped['New_tubing'] = np.sign(features_grouped['New_tubing'].astype(int))
features_grouped['Spiral_rod_guides'] = np.sign(features_grouped['Spiral_rod_guides'].astype(int))
features_grouped['Heavy_wall_barrel'] = np.sign(features_grouped['Heavy_wall_barrel'].astype(int))
features_grouped['Gas_separator'] = np.sign(features_grouped['Gas_separator'].astype(int))
features_grouped['Brass_barrel'] = np.sign(features_grouped['Brass_barrel'].astype(int))
features_grouped['FARR_plunger'] = np.sign(features_grouped['FARR_plunger'].astype(int))
features_grouped['Tubing_anchored'] = np.sign(features_grouped['Tubing_anchored'].astype(int))
features_grouped['Setting_depth'] = features_grouped['ABSETZTEUFE']
features_grouped['Pump_bore'] = features_grouped['API_KURZBEZ_TEIL1'].astype(int)
features_grouped['Stroke_length'] = features_grouped['HUBLAENGE']
features_grouped['Pumping_speed'] = features_grouped['HUBZAHL']
features_grouped['DLS'] = features_grouped['DOGLEG']
features_grouped['Full_flow_cages'] = np.sign(features_grouped['Full Flow Cages'].astype(int))
features_grouped['Mod_valve_rod_bearing'] = np.sign(features_grouped['Modifizierte \
Kolbenstangenführung mit Insert'].astype(int))
features_grouped['GLB'] = np.sign(features_grouped['Gas Lock Breaker'].astype(int))
features_grouped['Closed_installation'] = np.sign(features_grouped['geschlossene Installation'].astype(int))
features_grouped['Collet_valve_rod_bushing'] = np.sign(features_grouped['Collet/Lock Type Valve Rod Bushing'].astype(int))

features_grouped['FARR_plunger'] = features_grouped['FARR_plunger'].fillna(0)

features_grouped = features_grouped.drop(columns =['Conslot Filter', 'Stanley Filter', 
                                                  'neu', 'gebr.', 'weiß', 'gelb',
                                                   'Full Flow Cages', 'Gas Lock Breaker',
                                                   'geschlossene Installation',
                                                   'Collet/Lock Type Valve Rod Bushing',
                                                   'Modifizierte Kolbenstangenführung mit Insert',
                                                  'Spiral Rod Guide 2 7/8"', 'Spiral Rod Guide 3 1/2"',
                                                  'RHAC', 'RHBM', 'RWAC', 'TH', 'DOGLEG',
                                                   'Mantelrohr 2 7/8"', 'Mantelrohr 3 1/2"',
                                                  'La Bomba Memory Gauge', 'Gasanker',
                                                   'Heavy_cages_balls',
                                                   'Gasseparator - PCM', 'A4 Ventilkugeln',
                                                  'A6 Ventilkugeln','Messingbarrel', 'ABSETZTEUFE',
                                                  'API_KURZBEZ_TEIL1', 'HUBLAENGE', 'HUBZAHL',
                                                   'Steigrohranker', 'LSP ohne Gummi',
                                                   'TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten',
                                                  '!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten'])

features_grouped = features_grouped.drop_duplicates()
features_grouped = sqldf("SELECT * FROM features_grouped WHERE dls < 10")
features_grouped = features_grouped.drop(columns = ['DATUM_1','SONDENKURZNAME_1'])
features_grouped = features_grouped.copy()
features_grouped

In [ ]:
mean = features_grouped['RUNLIFE'].mean()
variance = features_grouped['RUNLIFE'].var()

print(mean)
print(variance)

In [ ]:
alpha = (variance-mean)/(mean**2)
print(alpha)

In [ ]:
features_grouped.sum()

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats



mask = np.random.rand(len(features_grouped)) < 0.9
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + Pumping_speed + Pump_bore + New_tubing + Sand_filter + A4_valve_balls \
+ A6_valve_balls + Tubing_anchored + Gas_separator + Spiral_rod_guides + Stroke_length + \
FARR_plunger + New_tubing + Desander + Mod_valve_rod_bearing + Collet_valve_rod_bushing + GLB + Tubing_pump +\
Closed_installation + Full_flow_cages + Desander + Jacket_pipe + Brass_barrel"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

fig, ax = plt.subplots()

ax.scatter(results.predict(X_train), results.resid_pearson)
ax.hlines(0, 0, 1)
ax.set_ylabel('Pearson Residuals')
ax.set_xlabel('Fitted values')

results.predict(X_train).var()

residual_deviance = results.deviance
degrees_of_freedom = results.df_resid

chi_squared_statistic = residual_deviance 
p_value = 1 - stats.chi2.cdf(chi_squared_statistic, degrees_of_freedom)

print(p_value)

print(results.summary().as_latex())

In [ ]:
def features_univariate(feature):
    model_features = "RUNLIFE ~ " + feature
    return model_features
    
columns = features_grouped.drop(columns='RUNLIFE').columns

for i in range(columns.size):
    y_train, X_train = dmatrices(features_univariate(columns[i]), df_train, return_type='dataframe')
    y_test, X_test = dmatrices(features_univariate(columns[i]), df_test, return_type='dataframe')

    training_results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial()).fit()
    
    print(training_results.summary())
   
    plt.xlim(0,2000)
    plt.ylim(0,2000)
    plt.scatter(y_test,training_results.predict(X_test))
    plt.show()
    
    
    plt.xlim(0,2000)
    plt.ylim(0,2000)
    plt.scatter(y_train,training_results.predict(X_train))
    plt.show()
    
    plt.scatter(training_results.predict(X_train), results.resid_pearson)
    plt.ylabel('Pearson Residuals')
    plt.xlabel('Fitted values')
    plt.show()

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt

mask = np.random.rand(len(features_grouped)) < 0.8
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + Pumping_speed + Pump_bore + \
Gas_separator + Spiral_rod_guides + Stroke_length +  GLB + Tubing_pump + \
Closed_installation + Jacket_pipe + Full_flow_cages"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.scatter(y_test,results.predict(X_test))
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.show()

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.scatter(y_train,results.predict(X_train))
plt.show()

plt.scatter(results.predict(X_train), results.resid_pearson)
plt.ylabel('Pearson residuals')
plt.xlabel('Fitted values')
plt.show()

results.predict(X_train).var()

residual_deviance = results.deviance
degrees_of_freedom = results.df_resid

chi_squared_statistic = residual_deviance
p_value = 1 - stats.chi2.cdf(chi_squared_statistic, degrees_of_freedom)

print(p_value)

print(results.summary().as_latex())

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt

mask = np.random.rand(len(features_grouped)) < 0.8
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~  Setting_depth + \
GLB + \
Closed_installation + Jacket_pipe "

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.scatter(y_test,results.predict(X_test))
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.show()

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.scatter(y_train,results.predict(X_train))
plt.show()

plt.scatter(results.predict(X_train), results.resid_pearson)
plt.ylabel('Pearson residuals')
plt.xlabel('Fitted values')
plt.show()

results.predict(X_train).var()

residual_deviance = results.deviance
degrees_of_freedom = results.df_resid

chi_squared_statistic = residual_deviance
p_value = 1 - stats.chi2.cdf(chi_squared_statistic, degrees_of_freedom)

print(p_value)

print(results.summary().as_latex())

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt

mask = np.random.rand(len(features_grouped)) < 0.9
df_train = features_grouped[mask]
df_test = features_grouped[~mask]


print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + Pumping_speed + Pump_bore + New_tubing + Sand_filter + A4_valve_balls \
+ A6_valve_balls + Tubing_anchored + Gas_separator + Spiral_rod_guides + Stroke_length + \
FARR_plunger + New_tubing + Desander + Mod_valve_rod_bearing + Collet_valve_rod_bushing + GLB + Tubing_pump +\
Closed_installation + Full_flow_cages + Desander + Jacket_pipe"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8).fit()

print(results.summary())

fig, ax = plt.subplots()

ax.scatter(results.predict(X_train), results.resid_anscombe)
ax.hlines(0, 0, 1)
ax.set_ylabel('Anscombe Residuals')
ax.set_xlabel('Fitted values')

results.predict(X_train).var()

print(results.summary().as_latex())
 

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt

mask = np.random.rand(len(features_grouped)) < 0.8
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + Pumping_speed + Stroke_length + \
 Mod_valve_rod_bearing + Collet_valve_rod_bushing + GLB + Tubing_pump +\
Closed_installation + Full_flow_cages + Desander + Jacket_pipe + Gas_separator"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

plt.scatter(y_test,results.predict(X_test))
plt.xlim(0,1500)
plt.ylim(0,1500)

In [ ]:
pumpdurchmesser = data_tpschaden['API_KURZBEZ_TEIL1'].str[3:]
pumpentyp = pd.get_dummies(data_tpschaden['API_KURZBEZ_TEIL2'])
pgzustand = pd.get_dummies(data_tpschaden['PG_TYP2'])
tubingcondition = pd.get_dummies(data_tpschaden['TUB_TYP2'])
gasanker = pd.get_dummies(data_tpschaden['GAS_ANKER'])
packer = pd.get_dummies(data_tpschaden['PACK_ANKER'])
hubzahl = data_tpschaden['HUBZAHL']
hublaenge = data_tpschaden['HUBLAENGE']
farr = data_tpschaden['SPEZ_FARR'].fillna(0).astype(int)
spez1 = pd.get_dummies(data_tpschaden['SPEZ_EQUI1'])
spez2 = pd.get_dummies(data_tpschaden['SPEZ_EQUI2'])
spez3 = pd.get_dummies(data_tpschaden['SPEZ_EQUI3'])
spez4 = pd.get_dummies(data_tpschaden['SPEZ_EQUI4'])
spez5 = pd.get_dummies(data_tpschaden['SPEZ_EQUI5'])
spez6 = pd.get_dummies(data_tpschaden['SPEZ_EQUI6'])

features = pd.concat([data_tpschaden['SONDENKURZNAME_1'], data_tpschaden['DATUM_1'], data_tpschaden['RUNLIFE'].astype(int),data_tpschaden['DOGLEG'],data_tpschaden['ABSETZTEUFE'],data_tpschaden['HUBZAHL'],
                      data_tpschaden['HUBLAENGE'], pumpdurchmesser, pumpentyp, pgzustand, 
                      tubingcondition, gasanker, packer, spez1, spez2,
                      spez3, spez4, spez5, spez6], axis = 1)


features_grouped = features.groupby(features.columns, axis=1).sum()


farr_heavycage_brass = features_grouped['TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten']
heavycage = features_grouped['!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten']

features_grouped['Tubing_pump'] = features_grouped['TH']
features_grouped['Sand_filter'] = features_grouped['Stanley Filter']
features_grouped['New_tubing'] = features_grouped['neu'] + features_grouped['gelb']
features_grouped['Spiral_rod_guides'] = features_grouped['Spiral Rod Guide 2 7/8"'] + features_grouped['Spiral Rod Guide 3 1/2"']
features_grouped['Heavy_wall_barrel'] = features_grouped['RHAC'] + features_grouped['RHBM'] + features_grouped['TH']
features_grouped['Jacket_pipe'] = features_grouped['Mantelrohr 2 7/8"'] + features_grouped['Mantelrohr 3 1/2"']
features_grouped['Gas_separator'] = features_grouped['Gasanker'] + features_grouped['Gasseparator - PCM']
features_grouped['Brass_barrel'] = features_grouped['Messingbarrel'] + farr_heavycage_brass
features_grouped['FARR_plunger'] = farr + farr_heavycage_brass
features_grouped['Heavy_cages_balls'] = heavycage + farr_heavycage_brass
features_grouped['Tubing_anchored'] = features_grouped['Steigrohranker'] + features_grouped['LSP ohne Gummi']
features_grouped['A4_valve_balls'] = features_grouped['A4 Ventilkugeln'] + features_grouped['Heavy_cages_balls']

features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4_valve_balls'])
features_grouped['A6_valve_balls'] = np.sign(features_grouped['A6 Ventilkugeln'])
features_grouped['Jacket_pipe'] = np.sign(features_grouped['Jacket_pipe'])
features_grouped['Sand_filter'] = np.sign(features_grouped['Sand_filter'].astype(int))
features_grouped['New_tubing'] = np.sign(features_grouped['New_tubing'].astype(int))
features_grouped['Spiral_rod_guides'] = np.sign(features_grouped['Spiral_rod_guides'].astype(int))
features_grouped['Heavy_wall_barrel'] = np.sign(features_grouped['Heavy_wall_barrel'].astype(int))
features_grouped['Gas_separator'] = np.sign(features_grouped['Gas_separator'].astype(int))
features_grouped['Brass_barrel'] = np.sign(features_grouped['Brass_barrel'].astype(int))
features_grouped['FARR_plunger'] = np.sign(features_grouped['FARR_plunger'].astype(int))
features_grouped['Tubing_anchored'] = np.sign(features_grouped['Tubing_anchored'].astype(int))
features_grouped['Setting_depth'] = features_grouped['ABSETZTEUFE']
features_grouped['Pump_bore'] = features_grouped['API_KURZBEZ_TEIL1'].astype(int)
features_grouped['Stroke_length'] = features_grouped['HUBLAENGE']
features_grouped['Pumping_speed'] = features_grouped['HUBZAHL']
features_grouped['DLS'] = features_grouped['DOGLEG']
features_grouped['Full_flow_cages'] = np.sign(features_grouped['Full Flow Cages'].astype(int))
features_grouped['Mod_valve_rod_bearing'] = np.sign(features_grouped['Modifizierte \
Kolbenstangenführung mit Insert'].astype(int))
features_grouped['GLB'] = np.sign(features_grouped['Gas Lock Breaker'].astype(int))
features_grouped['Closed_installation'] = np.sign(features_grouped['geschlossene Installation'].astype(int))
features_grouped['Collet_valve_rod_bushing'] = np.sign(features_grouped['Collet/Lock Type Valve Rod Bushing'].astype(int))


features_grouped['FARR_plunger'] = features_grouped['FARR_plunger'].fillna(0)

features_grouped = features_grouped.drop(columns =['Stanley Filter',
                                                  'neu', 'gebr.', 'weiß', 'gelb',
                                                   'Full Flow Cages', 'Gas Lock Breaker',
                                                   'geschlossene Installation',
                                                   'Heavy_cages_balls',
                                                   'Collet/Lock Type Valve Rod Bushing',
                                                   'Modifizierte Kolbenstangenführung mit Insert',
                                                  'Spiral Rod Guide 2 7/8"', 'Spiral Rod Guide 3 1/2"',
                                                  'RHAC', 'RHBM', 'RWAC', 'TH', 'DOGLEG',
                                                   'Mantelrohr 2 7/8"', 'Mantelrohr 3 1/2"',
                                                  'La Bomba Memory Gauge', 'Gasanker',
                                                   'Gasseparator - PCM', 'A4 Ventilkugeln',
                                                  'A6 Ventilkugeln','Messingbarrel', 'ABSETZTEUFE',
                                                  'API_KURZBEZ_TEIL1', 'HUBLAENGE', 'HUBZAHL',
                                                   'Steigrohranker', 'LSP ohne Gummi',
                                                   'TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten',
                                                  '!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten'])

features_grouped = features_grouped.drop_duplicates()
features_grouped = sqldf("SELECT * FROM features_grouped WHERE dls < 10")
features_grouped = features_grouped.drop(columns = ['DATUM_1','SONDENKURZNAME_1'])
features_grouped = features_grouped.copy()
features_grouped

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats



mask = np.random.rand(len(features_grouped)) < 0.9
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + Pumping_speed + Pump_bore + New_tubing + Sand_filter + A4_valve_balls \
+ A6_valve_balls + Tubing_anchored + Gas_separator + Spiral_rod_guides + Stroke_length + \
FARR_plunger + New_tubing + Desander + Mod_valve_rod_bearing + Collet_valve_rod_bushing + GLB + Tubing_pump +\
Closed_installation + Full_flow_cages + Desander + Jacket_pipe + Brass_barrel"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

fig, ax = plt.subplots()

ax.scatter(results.predict(X_train), results.resid_pearson)
ax.hlines(0, 0, 1)
ax.set_ylabel('Pearson Residuals')
ax.set_xlabel('Fitted values')

results.predict(X_train).var()

residual_deviance = results.deviance
degrees_of_freedom = results.df_resid

chi_squared_statistic = residual_deviance 
p_value = 1 - stats.chi2.cdf(chi_squared_statistic, degrees_of_freedom)

print(p_value)

print(results.summary().as_latex())

In [ ]:
def features_univariate(feature):
    model_features = "RUNLIFE ~ " + feature
    return model_features
    
columns = features_grouped.drop(columns='RUNLIFE').columns

for i in range(columns.size):
    y_train, X_train = dmatrices(features_univariate(columns[i]), df_train, return_type='dataframe')
    y_test, X_test = dmatrices(features_univariate(columns[i]), df_test, return_type='dataframe')

    training_results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial()).fit()
    
    print(training_results.summary())
   
    plt.xlim(0,2000)
    plt.ylim(0,2000)
    plt.scatter(y_test,training_results.predict(X_test))
    plt.show()
    
    
    plt.xlim(0,2000)
    plt.ylim(0,2000)
    plt.scatter(y_train,training_results.predict(X_train))
    plt.show()
    
    plt.scatter(training_results.predict(X_train), results.resid_pearson)
    plt.ylabel('Pearson Residuals')
    plt.xlabel('Fitted values')
    plt.show()

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats



mask = np.random.rand(len(features_grouped)) < 0.9
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + Pumping_speed + New_tubing + \
Spiral_rod_guides + Stroke_length + Jacket_pipe + Gas_separator + Closed_installation + \
Brass_barrel +  Pump_bore + Gas_separator + \
+ Jacket_pipe"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

results.predict(X_train).var()

residual_deviance = results.deviance
degrees_of_freedom = results.df_resid

chi_squared_statistic = residual_deviance 
p_value = 1 - stats.chi2.cdf(chi_squared_statistic, degrees_of_freedom)

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.scatter(y_test,results.predict(X_test))
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.show()

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.scatter(y_train,results.predict(X_train))
plt.show()

plt.scatter(results.predict(X_train), results.resid_pearson)
plt.ylabel('Pearson residuals')
plt.xlabel('Fitted values')
plt.show()

print(p_value)

print(results.summary().as_latex())

In [ ]:
pumpdurchmesser = data_undicht['API_KURZBEZ_TEIL1'].str[3:]
pumpentyp = pd.get_dummies(data_undicht['API_KURZBEZ_TEIL2'])
pgzustand = pd.get_dummies(data_undicht['PG_TYP2'])
tubingcondition = pd.get_dummies(data_undicht['TUB_TYP2'])
gasanker = pd.get_dummies(data_undicht['GAS_ANKER'])
packer = pd.get_dummies(data_undicht['PACK_ANKER'])
hubzahl = data_undicht['HUBZAHL']
hublaenge = data_undicht['HUBLAENGE']
farr = data_undicht['SPEZ_FARR'].fillna(0).astype(int)
spez1 = pd.get_dummies(data_undicht['SPEZ_EQUI1'])
spez2 = pd.get_dummies(data_undicht['SPEZ_EQUI2'])
spez3 = pd.get_dummies(data_undicht['SPEZ_EQUI3'])
spez4 = pd.get_dummies(data_undicht['SPEZ_EQUI4'])
spez5 = pd.get_dummies(data_undicht['SPEZ_EQUI5'])
spez6 = pd.get_dummies(data_undicht['SPEZ_EQUI6'])

features = pd.concat([data_undicht['SONDENKURZNAME_1'], data_undicht['DATUM_1'], data_undicht['RUNLIFE'].astype(int),data_undicht['DOGLEG'],data_undicht['ABSETZTEUFE'],data_undicht['HUBZAHL'],
                      data_undicht['HUBLAENGE'], pumpdurchmesser, pumpentyp, pgzustand, 
                      tubingcondition, gasanker, packer, spez1, spez2,
                      spez3, spez4, spez5, spez6], axis = 1)


features_grouped = features.groupby(features.columns, axis=1).sum()


farr_heavycage_brass = features_grouped['TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten']
heavycage = features_grouped['!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten']

features_grouped['Tubing_pump'] = features_grouped['TH']
features_grouped['New_tubing'] = features_grouped['neu'] + features_grouped['gelb']
features_grouped['Spiral_rod_guides'] = features_grouped['Spiral Rod Guide 2 7/8"'] + features_grouped['Spiral Rod Guide 3 1/2"']
features_grouped['Heavy_wall_barrel'] = features_grouped['RHAC'] + features_grouped['RHBM'] + features_grouped['TH']
features_grouped['Jacket_pipe'] = features_grouped['Mantelrohr 2 7/8"'] + features_grouped['Mantelrohr 3 1/2"']
features_grouped['Gas_separator'] = features_grouped['Gasanker'] + features_grouped['Gasseparator - PCM']
features_grouped['Brass_barrel'] = features_grouped['Messingbarrel'] + farr_heavycage_brass
features_grouped['FARR_plunger'] = farr + farr_heavycage_brass
features_grouped['Heavy_cages_balls'] = heavycage + farr_heavycage_brass
features_grouped['Tubing_anchored'] = features_grouped['Steigrohranker'] + features_grouped['LSP ohne Gummi']
features_grouped['A4_valve_balls'] = features_grouped['A4 Ventilkugeln'] + features_grouped['Heavy_cages_balls']

features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4_valve_balls'])
features_grouped['A6_valve_balls'] = np.sign(features_grouped['A6 Ventilkugeln'])
features_grouped['Jacket_pipe'] = np.sign(features_grouped['Jacket_pipe'].astype(int))
features_grouped['New_tubing'] = np.sign(features_grouped['New_tubing'].astype(int))
features_grouped['Spiral_rod_guides'] = np.sign(features_grouped['Spiral_rod_guides'].astype(int))
features_grouped['Heavy_wall_barrel'] = np.sign(features_grouped['Heavy_wall_barrel'].astype(int))
features_grouped['Gas_separator'] = np.sign(features_grouped['Gas_separator'].astype(int))
features_grouped['Brass_barrel'] = np.sign(features_grouped['Brass_barrel'].astype(int))
features_grouped['FARR_plunger'] = np.sign(features_grouped['FARR_plunger'].astype(int))
features_grouped['Tubing_anchored'] = np.sign(features_grouped['Tubing_anchored'].astype(int))
features_grouped['Setting_depth'] = features_grouped['ABSETZTEUFE']
features_grouped['Pump_bore'] = features_grouped['API_KURZBEZ_TEIL1'].astype(int)
features_grouped['Stroke_length'] = features_grouped['HUBLAENGE']
features_grouped['Pumping_speed'] = features_grouped['HUBZAHL']
features_grouped['DLS'] = features_grouped['DOGLEG']
features_grouped['Full_flow_cages'] = np.sign(features_grouped['Full Flow Cages'].astype(int))
features_grouped['Mod_valve_rod_bearing'] = np.sign(features_grouped['Modifizierte \
Kolbenstangenführung mit Insert'].astype(int))
features_grouped['GLB'] = np.sign(features_grouped['Gas Lock Breaker'].astype(int))
features_grouped['Closed_installation'] = np.sign(features_grouped['geschlossene Installation'].astype(int))
features_grouped['Collet_valve_rod_bushing'] = np.sign(features_grouped['Collet/Lock Type Valve Rod Bushing'].astype(int))


features_grouped['FARR_plunger'] = features_grouped['FARR_plunger'].fillna(0)

features_grouped = features_grouped.drop(columns =['neu', 'gebr.', 'weiß', 'gelb',
                                                  'Spiral Rod Guide 2 7/8"', 'Spiral Rod Guide 3 1/2"',
                                                   'Full Flow Cages', 'Gas Lock Breaker',
                                                   'geschlossene Installation',
                                                    'Heavy_cages_balls',
                                                   'Collet/Lock Type Valve Rod Bushing',
                                                   'Modifizierte Kolbenstangenführung mit Insert',
                                                  'RHAC', 'RHBM', 'RWAC', 'TH', 'DOGLEG',
                                                   'Mantelrohr 2 7/8"', 'Mantelrohr 3 1/2"',
                                                  'La Bomba Memory Gauge', 'Gasanker',
                                                   'Gasseparator - PCM', 'A4 Ventilkugeln',
                                                  'A6 Ventilkugeln','Messingbarrel', 'ABSETZTEUFE',
                                                  'API_KURZBEZ_TEIL1', 'HUBLAENGE', 'HUBZAHL',
                                                   'Steigrohranker', 'LSP ohne Gummi',
                                                   'TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten',
                                                  '!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten'])

features_grouped = features_grouped.drop_duplicates()
features_grouped = features_grouped.drop_duplicates()
features_grouped = sqldf("SELECT * FROM features_grouped WHERE dls < 10")
features_grouped = features_grouped.drop(columns = ['DATUM_1','SONDENKURZNAME_1'])
features_grouped = features_grouped.copy()
features_grouped

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats



mask = np.random.rand(len(features_grouped)) < 0.9
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + Pumping_speed + Pump_bore + New_tubing + A4_valve_balls \
+ A6_valve_balls + Tubing_anchored + Gas_separator + Spiral_rod_guides + Stroke_length + \
FARR_plunger + New_tubing + Desander + Mod_valve_rod_bearing + Collet_valve_rod_bushing + GLB + Tubing_pump +\
Closed_installation + Full_flow_cages + Desander + Jacket_pipe + Brass_barrel"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

fig, ax = plt.subplots()

ax.scatter(results.predict(X_train), results.resid_pearson)
ax.hlines(0, 0, 1)
ax.set_ylabel('Pearson Residuals')
ax.set_xlabel('Fitted values')

results.predict(X_train).var()

residual_deviance = results.deviance
degrees_of_freedom = results.df_resid

chi_squared_statistic = residual_deviance  
p_value = 1 - stats.chi2.cdf(chi_squared_statistic, degrees_of_freedom)

print(p_value)

print(results.summary().as_latex())

In [ ]:
def features_univariate(feature):
    model_features = "RUNLIFE ~ " + feature
    return model_features
    
columns = features_grouped.drop(columns='RUNLIFE').columns

for i in range(columns.size):
    y_train, X_train = dmatrices(features_univariate(columns[i]), df_train, return_type='dataframe')
    y_test, X_test = dmatrices(features_univariate(columns[i]), df_test, return_type='dataframe')

    training_results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial()).fit()
    
    print(training_results.summary())
   
    plt.xlim(0,2000)
    plt.ylim(0,2000)
    plt.scatter(y_test,training_results.predict(X_test))
    plt.show()
    
    
    plt.xlim(0,2000)
    plt.ylim(0,2000)
    plt.scatter(y_train,training_results.predict(X_train))
    plt.show()
    
    plt.scatter(training_results.predict(X_train), results.resid_pearson)
    plt.ylabel('Pearson Residuals')
    plt.xlabel('Fitted values')
    plt.show()

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats



mask = np.random.rand(len(features_grouped)) < 0.9
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + Pumping_speed + \
Spiral_rod_guides + Stroke_length + Jacket_pipe + Gas_separator + \
Brass_barrel +  Pump_bore + GLB + \
Closed_installation + Jacket_pipe"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

results.predict(X_train).var()

residual_deviance = results.deviance
degrees_of_freedom = results.df_resid

chi_squared_statistic = residual_deviance
p_value = 1 - stats.chi2.cdf(chi_squared_statistic, degrees_of_freedom)

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.scatter(y_test,results.predict(X_test))
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.show()

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.scatter(y_train,results.predict(X_train))
plt.show()

plt.scatter(results.predict(X_train), results.resid_pearson)
plt.ylabel('Pearson residuals')
plt.xlabel('Fitted values')
plt.show()

print(p_value)

print(results.summary().as_latex())

In [ ]:
pumpdurchmesser = data_pgbruch['API_KURZBEZ_TEIL1'].str[3:]
pumpentyp = pd.get_dummies(data_pgbruch['API_KURZBEZ_TEIL2'])
pgzustand = pd.get_dummies(data_pgbruch['PG_TYP2'])
tubingcondition = pd.get_dummies(data_pgbruch['TUB_TYP2'])
gasanker = pd.get_dummies(data_pgbruch['GAS_ANKER'])
packer = pd.get_dummies(data_pgbruch['PACK_ANKER'])
hubzahl = data_pgbruch['HUBZAHL']
hublaenge = data_pgbruch['HUBLAENGE']
farr = data_pgbruch['SPEZ_FARR'].fillna(0).astype(int)
spez1 = pd.get_dummies(data_pgbruch['SPEZ_EQUI1'])
spez2 = pd.get_dummies(data_pgbruch['SPEZ_EQUI2'])
spez3 = pd.get_dummies(data_pgbruch['SPEZ_EQUI3'])
spez4 = pd.get_dummies(data_pgbruch['SPEZ_EQUI4'])
spez5 = pd.get_dummies(data_pgbruch['SPEZ_EQUI5'])
spez6 = pd.get_dummies(data_pgbruch['SPEZ_EQUI6'])

features = pd.concat([data_pgbruch['SONDENKURZNAME_1'], data_pgbruch['DATUM_1'], data_pgbruch['RUNLIFE'].astype(int),data_pgbruch['DOGLEG'],data_pgbruch['ABSETZTEUFE'],data_pgbruch['HUBZAHL'],
                      data_pgbruch['HUBLAENGE'], pumpdurchmesser, pumpentyp, pgzustand, 
                      tubingcondition, gasanker, packer, spez1, spez2,
                      spez3, spez4, spez5, spez6], axis = 1)


features_grouped = features.groupby(features.columns, axis=1).sum()


farr_heavycage_brass = features_grouped['TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten']
heavycage = features_grouped['!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten']

features_grouped['Tubing_pump'] = features_grouped['TH']
features_grouped['Sand_filter'] = features_grouped['Conslot Filter']
features_grouped['New_tubing'] = features_grouped['neu'] + features_grouped['gelb']
features_grouped['Spiral_rod_guides'] = features_grouped['Spiral Rod Guide 2 7/8"'] + features_grouped['Spiral Rod Guide 3 1/2"']
features_grouped['Heavy_wall_barrel'] = features_grouped['RHAC'] + features_grouped['RHBM'] + features_grouped['TH']
features_grouped['Jacket_pipe'] = features_grouped['Mantelrohr 2 7/8"'] + features_grouped['Mantelrohr 3 1/2"']
features_grouped['Gas_separator'] = features_grouped['Gasanker'] + features_grouped['Gasseparator - PCM']
features_grouped['Brass_barrel'] = features_grouped['Messingbarrel'] + farr_heavycage_brass
features_grouped['FARR_plunger'] = farr + farr_heavycage_brass
features_grouped['Heavy_cages_balls'] = heavycage + farr_heavycage_brass
features_grouped['Tubing_anchored'] = features_grouped['Steigrohranker'] + features_grouped['LSP ohne Gummi']
features_grouped['A4_valve_balls'] = features_grouped['A4 Ventilkugeln'] + features_grouped['Heavy_cages_balls']

features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4_valve_balls'])
features_grouped['Sand_filter'] = np.sign(features_grouped['Sand_filter'].astype(int))
features_grouped['Jacket_pipe'] = np.sign(features_grouped['Jacket_pipe'].astype(int))
features_grouped['New_tubing'] = np.sign(features_grouped['New_tubing'].astype(int))
features_grouped['Spiral_rod_guides'] = np.sign(features_grouped['Spiral_rod_guides'].astype(int))
features_grouped['Heavy_wall_barrel'] = np.sign(features_grouped['Heavy_wall_barrel'].astype(int))
features_grouped['Gas_separator'] = np.sign(features_grouped['Gas_separator'].astype(int))
features_grouped['Brass_barrel'] = np.sign(features_grouped['Brass_barrel'].astype(int))
features_grouped['FARR_plunger'] = np.sign(features_grouped['FARR_plunger'].astype(int))
features_grouped['Tubing_anchored'] = np.sign(features_grouped['Tubing_anchored'].astype(int))
features_grouped['Setting_depth'] = features_grouped['ABSETZTEUFE']
features_grouped['Pump_bore'] = features_grouped['API_KURZBEZ_TEIL1'].astype(int)
features_grouped['Stroke_length'] = features_grouped['HUBLAENGE']
features_grouped['Pumping_speed'] = features_grouped['HUBZAHL']
features_grouped['DLS'] = features_grouped['DOGLEG']
features_grouped['Full_flow_cages'] = np.sign(features_grouped['Full Flow Cages'].astype(int))
features_grouped['Mod_valve_rod_bearing'] = np.sign(features_grouped['Modifizierte \
Kolbenstangenführung mit Insert'].astype(int))
features_grouped['GLB'] = np.sign(features_grouped['Gas Lock Breaker'].astype(int))
features_grouped['Closed_installation'] = np.sign(features_grouped['geschlossene Installation'].astype(int))


features_grouped['FARR_plunger'] = features_grouped['FARR_plunger'].fillna(0)

features_grouped = features_grouped.drop(columns =['Conslot Filter',
                                                  'neu', 'gebr.', 'weiß', 'gelb',
                                                   'Full Flow Cages', 'Gas Lock Breaker',
                                                   'geschlossene Installation',
                                                   'Heavy_cages_balls',
                                                   'Modifizierte Kolbenstangenführung mit Insert',
                                                  'Spiral Rod Guide 2 7/8"', 'Spiral Rod Guide 3 1/2"',
                                                  'RHAC', 'RHBM', 'TH', 'DOGLEG',
                                                   'Mantelrohr 2 7/8"', 'Mantelrohr 3 1/2"',
                                                  'La Bomba Memory Gauge', 'Gasanker',
                                                   'Gasseparator - PCM', 'A4 Ventilkugeln',
                                                  'Messingbarrel', 'ABSETZTEUFE',
                                                  'API_KURZBEZ_TEIL1', 'HUBLAENGE', 'HUBZAHL',
                                                   'Steigrohranker', 'LSP ohne Gummi',
                                                   'TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten',
                                                  '!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten'])

features_grouped = features_grouped.drop_duplicates()
features_grouped = features_grouped.drop_duplicates()
features_grouped = sqldf("SELECT * FROM features_grouped WHERE dls < 10")
features_grouped = features_grouped.drop(columns = ['DATUM_1','SONDENKURZNAME_1'])
features_grouped = features_grouped.copy()
features_grouped

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats



mask = np.random.rand(len(features_grouped)) < 0.9
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + Pumping_speed + Pump_bore + New_tubing + Sand_filter + Tubing_anchored + Gas_separator + Spiral_rod_guides + Stroke_length + \
FARR_plunger + New_tubing + Desander + Mod_valve_rod_bearing + GLB + Tubing_pump +\
Closed_installation + Full_flow_cages + Desander + Jacket_pipe + Brass_barrel"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

fig, ax = plt.subplots()

ax.scatter(results.predict(X_train), results.resid_pearson)
ax.hlines(0, 0, 1)
ax.set_ylabel('Pearson Residuals')
ax.set_xlabel('Fitted values')

results.predict(X_train).var()

residual_deviance = results.deviance
degrees_of_freedom = results.df_resid

chi_squared_statistic = residual_deviance  
p_value = 1 - stats.chi2.cdf(chi_squared_statistic, degrees_of_freedom)

print(p_value)

print(results.summary().as_latex())

In [ ]:
def features_univariate(feature):
    model_features = "RUNLIFE ~ " + feature
    return model_features
    
columns = features_grouped.drop(columns='RUNLIFE').columns

for i in range(columns.size):
    y_train, X_train = dmatrices(features_univariate(columns[i]), df_train, return_type='dataframe')
    y_test, X_test = dmatrices(features_univariate(columns[i]), df_test, return_type='dataframe')

    training_results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial()).fit()
    
    print(training_results.summary())
   
    plt.xlim(0,2000)
    plt.ylim(0,2000)
    plt.scatter(y_test,training_results.predict(X_test))
    plt.show()
    
    
    plt.xlim(0,2000)
    plt.ylim(0,2000)
    plt.scatter(y_train,training_results.predict(X_train))
    plt.show()
    
    plt.scatter(training_results.predict(X_train), results.resid_pearson)
    plt.ylabel('Pearson Residuals')
    plt.xlabel('Fitted values')
    plt.show()

In [ ]:
from patsy import dmatrices
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats



mask = np.random.rand(len(features_grouped)) < 0.9
df_train = features_grouped[mask]
df_test = features_grouped[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

model_features = "RUNLIFE ~ DLS + Setting_depth + + Pump_bore+ Pumping_speed + New_tubing + \
Tubing_anchored + Spiral_rod_guides + Stroke_length + \
New_tubing + Mod_valve_rod_bearing +\
Closed_installation + Jacket_pipe"

y_train, X_train = dmatrices(model_features, df_train, return_type='dataframe')
y_test, X_test = dmatrices(model_features, df_test, return_type='dataframe')

results = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha = 0.8)).fit()

print(results.summary())

results.predict(X_train).var()

residual_deviance = results.deviance
degrees_of_freedom = results.df_resid

chi_squared_statistic = residual_deviance 
p_value = 1 - stats.chi2.cdf(chi_squared_statistic, degrees_of_freedom)

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.scatter(y_test,results.predict(X_test))
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.show()

plt.xlim(0,2000)
plt.ylim(0,2000)
plt.xlabel('Observed runlife')
plt.ylabel('Predicted runlife')
plt.scatter(y_train,results.predict(X_train))
plt.show()

plt.scatter(results.predict(X_train), results.resid_pearson)
plt.ylabel('Pearson residuals')
plt.xlabel('Fitted values')
plt.show()

print(p_value)

print(results.summary().as_latex())